In [ ]:
!pip install selenium

In [ ]:
!pip install beautifulsoup4

In [ ]:
pip install selenium beautifulsoup4 webdriver-manager

In [ ]:
###############バージョン3#######################
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pandas as pd
import re

# Seleniumセットアップ
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriverの起動
service = Service(ChromeDriverManager().install())

# 日付変換関数
def convert_date_format_simple(date_str):
    """
    '/' で分割して '2024年2月22日' に変換する方法
    """
    parts = date_str.split("/")
    if len(parts) == 3:
        return f"{parts[0]}年{int(parts[1])}月{int(parts[2])}日"
    return date_str  # 変換できない場合はそのまま返す

#HTMLから株価データを抽出する関数
def extract_stock_data(html):
    """
    HTMLを解析し、日付・始値・高値・安値・終値を取得
    """
    soup = BeautifulSoup(html, "html.parser")

    day_stock_data = []

    for i in range(1, 6):
        element = soup.select_one(
            f'#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child({i})'
        )
        if element:
            st_data = element.get_text(strip=True)
        else:
            print(f"⚠️ データ取得失敗: tr:nth-child({i}) のデータが見つかりません")
            st_data = "N/A"

        # 日付のフォーマット変換
        if i == 1:
            try:
                chg_date = convert_date_format_simple(st_data)
                st_data = chg_date
            except ValueError:
                print(f"⚠️ 日付フォーマットエラー: {st_data}")
                st_data = "N/A"
        else:
            st_data = float(st_data[4:].replace(',', ''))

        day_stock_data.append(st_data)

    return day_stock_data

# Webページへアクセスし、データを抽出する関数
def get_stock_values(chrome_driver, url):
    chrome_driver.get(url)
    time.sleep(3)

    chart_element = chrome_driver.find_element(By.CSS_SELECTOR, "#highcharts-0 > svg > g:nth-child(6)")
    graph_width = chart_element.size['width']
    print(graph_width)

    # グラフの中央にマウスポインタを移動する
    chrome_actions = ActionChains(chrome_driver)
    chrome_actions.move_to_element(chart_element).perform()
    chrome_actions.move_by_offset(graph_width // 2, 0).perform()
    time.sleep(1)

    stock_data = []
    tmp_date=""

    #graph_width分左に1pxずつ移動して値を取得
    for _ in range(graph_width):
        chrome_actions.move_by_offset(-1, 0).perform()
        time.sleep(0.05)

        html = chrome_driver.page_source.encode('utf-8')
        data = extract_stock_data(html)

        if data[0]!=tmp_date:
            #print(data)
            stock_data.append(data)

        tmp_date=data[0]

    return stock_data

# --- 実行 ---
if __name__ == "__main__":
    # 日経平均株価のURL
    url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"

    # Google Chrome用のブラウザドライバーインスタンスを作成
    chrome_driver = webdriver.Chrome(options=chrome_options)

    # 時間の計測開始
    start_time = time.time()

    try:
        stock_data = get_stock_values(chrome_driver, url)

    finally:
        chrome_driver.quit()

    # スクレイピングの終了時間
    end_time = time.time()
    elapsed_time = end_time - start_time

    # データをデータフレーム化
    df = pd.DataFrame(stock_data, columns=["日付", "始値", "高値", "安値", "終値"])

    # データを出力# 結果の表示
    print(f"スクレイピング時間: {elapsed_time:.2f}秒")
    print(df)

559
スクレイピング時間: 367.36秒
             日付        始値        高値        安値        終値
0    2025年2月28日  37853.48  37925.28  36840.12  37155.50
1    2025年2月27日  38251.11  38369.96  38061.94  38256.17
2    2025年2月26日  38143.68  38175.24  37742.76  38142.37
3    2025年2月25日  38359.67  38485.95  38131.79  38237.79
4    2025年2月21日  38523.07  38808.81  38456.53  38776.94
..          ...       ...       ...       ...       ...
105  2024年9月20日  37714.07  37974.72  37654.11  37723.91
106  2024年9月19日  36958.93  37394.52  36958.93  37155.33
107  2024年9月18日  36546.04  36675.07  36127.45  36380.17
108  2024年9月17日  36601.93  36663.18  35828.54  36203.22
109  2024年9月13日  36882.41  36887.40  36441.34  36581.76

[110 rows x 5 columns]
